In [3]:
import matplotlib.pyplot as plt
from keras_nlp.api.models import DebertaV3Preprocessor
from keras_nlp.api.models import DebertaV3Classifier
import keras
import tensorflow as tf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl


def create_DebertaV3_model(model_preset, manual_preprocess=False):
    if not manual_preprocess:
        debertaV3 = DebertaV3Classifier.from_preset(model_preset, num_classes=6)
    else:
        debertaV3 = DebertaV3Classifier.from_preset(
            model_preset, preprocessor=None, num_classes=6
        )
    inputs = debertaV3.input
    outputs = debertaV3(inputs)

    prob_outputs = keras.layers.Activation("sigmoid")(outputs)
    model = keras.Model(inputs, prob_outputs)

    return model


def make_ordinary(y):
    n = len(y)
    z = np.zeros((n, 6), "float32")
    for i in range(n):
        s = y[i]
        z[i, :s] = 1
    return z

## Test model đã được lưu với đầu vào ngẫu nhiên

In [ ]:
test_df = pd.read_csv(".csv")

test_ds = (
    tf.data.Dataset.from_tensor_slices(
        (test_df["full_text"].tolist(),
          make_ordinary(test_df["score"].tolist()))
    )
    .cache()
    .batch(32)
)

model_test = create_DebertaV3_model("deberta_v3_extra_small_en")
model_test.load_weights("model.h5")
print("Loss in test set :")

### Một vài ví dụ

In [ ]:
df_sample = test_df.sample(n=10)
sample_ds = tf.data.Dataset.from_tensor_slices(
    (df_sample["full_text"].tolist(),
      make_ordinary(df_sample["score"].tolist()))
)
predicts = model_test.predict(test_ds, verbose=0)

for i in range(len(predicts)):
    predict = predicts[i]
    print("Essay: ", df_sample.iloc[i]["full_text"])
    print("Prediction: ", np.sum(predict >= 0.5))
    print("Truth Score: ", df_sample.iloc[i]["score"])